In [10]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import glob
import os

In [11]:
gt_data = pd.read_json(f"./images/configurations.json")

In [12]:
gt_data = (
    gt_data.T
)  # use indexx as a new column in the dataframe and called it image_path
gt_data = gt_data.reset_index()
gt_data = gt_data.rename(columns={"index": "image_path"})
gt_data

,image_path,num_points,num_intersections,line_colors,line_thickness
0,image_0_thickness_2.png,3,1,"[red, blue]",2
1,image_0_thickness_3.png,3,1,"[red, blue]",3
2,image_0_thickness_4.png,3,1,"[red, blue]",4
3,image_1_thickness_2.png,3,2,"[red, blue]",2
4,image_1_thickness_3.png,3,2,"[red, blue]",3
...,...,...,...,...,...
145,image_48_thickness_3.png,3,2,"[red, blue]",3
146,image_48_thickness_4.png,3,2,"[red, blue]",4
147,image_49_thickness_2.png,3,0,"[red, blue]",2
148,image_49_thickness_3.png,3,0,"[red, blue]",3


In [13]:
gt_data["model-output-file"] = gt_data["image_path"].apply(
    lambda x: "./images/"
    + x.replace(".png", "")
    + "-claude-3-sonnet-20240229-output.md"
)

gt_data["model-output-raw"] = gt_data["model-output-file"].apply(
    lambda x: (open(x, "r").read() if os.path.exists(os.path.join(x)) else None)
)

In [14]:
import re


def extract_marked_text(text):
    # Convert text to lowercase
    text = text.lower()

    # Regular expression to find numbers (digits or words)
    number_words = "zero|one|two|three|four|five|six|seven|eight|nine|ten"
    pattern = rf"\b({number_words}|\d+)\b"

    # Mapping from words to digits
    number_map = {
        "zero": "0",
        "one": "1",
        "two": "2",
        "three": "3",
        "four": "4",
        "five": "5",
        "six": "6",
        "seven": "7",
        "eight": "8",
        "nine": "9",
        "ten": "10",
    }

    # Search for the first occurrence of a number
    match = re.search(pattern, text)
    if match:
        # Map the word to its corresponding digit if it's a word
        return number_map.get(match.group(0), match.group(0))
    else:
        return "number_not_found"


gt_data["predicted"] = gt_data["model-output-raw"].apply(extract_marked_text)

In [15]:
gt_data["model-output-raw"]

0      In the given image, there is 1 intersection po...
1      The image shows two intersecting lines, one re...
2      In the given image, there is 1 intersection po...
3      In the given image, there are 2 line intersect...
4      In the given image, there are 2 line intersect...
                             ...                        
145    In the given image, there are 2 line intersect...
146    There are 2 line intersections in the provided...
147    In the image, there is 1 intersection point wh...
148    There is 1 line intersection or intersection p...
149    In the given image, there is 1 line intersecti...
Name: model-output-raw, Length: 150, dtype: object

In [16]:
gt_data["predicted"].value_counts()

predicted
1    65
2    59
4    24
3     2
Name: count, dtype: int64

In [17]:
cleaned_data = gt_data.copy()

In [18]:
# Ensure both columns are of integer type before comparison
cleaned_data["num_intersections"] = cleaned_data["num_intersections"].astype(int)
cleaned_data["predicted"] = cleaned_data["predicted"].astype(int)

cleaned_data["is_prediction_correct"] = (
    cleaned_data["num_intersections"] == cleaned_data["predicted"]
)
# Calculate accuracy
accuracy = cleaned_data["is_prediction_correct"].mean()
print(f"Overall Accuracy: {accuracy * 100:.2f}%")

Overall Accuracy: 34.00%


In [19]:
cleaned_data["line_thickness"] = cleaned_data["line_thickness"].astype(int)

# Calculate accuracy for each thickness
accuracy_by_thickness = cleaned_data.groupby("line_thickness").apply(
    lambda df: (df["is_prediction_correct"]).mean()
)

accuracy_by_thickness = 100 * accuracy_by_thickness.round(2)
accuracy_by_thickness

/var/folders/y1/b5wjnkp96_b7cvzncx0zsry40000gn/T/ipykernel_70697/2175508132.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  accuracy_by_thickness = cleaned_data.groupby("line_thickness").apply(


line_thickness
2    32.0
3    34.0
4    36.0
dtype: float64

In [20]:
print(accuracy_by_thickness.to_latex())

\begin{tabular}{lr}
\toprule
 & 0 \\
line_thickness &  \\
\midrule
2 & 32.000000 \\
3 & 34.000000 \\
4 & 36.000000 \\
\bottomrule
\end{tabular}



In [21]:
# show full data in dataframe
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
# show full content of each cell
pd.set_option("display.max_colwidth", None)

cleaned_data

,image_path,num_points,num_intersections,line_colors,line_thickness,model-output-file,model-output-raw,predicted,is_prediction_correct
0,image_0_thickness_2.png,3,1,"[red, blue]",2,./images/image_0_thickness_2-claude-3-sonnet-20240229-output.md,"In the given image, there is 1 intersection point where the red line and the blue line cross each other.",1,True
1,image_0_thickness_3.png,3,1,"[red, blue]",3,./images/image_0_thickness_3-claude-3-sonnet-20240229-output.md,"The image shows two intersecting lines, one red and one blue, that cross each other forming a single intersection point.",2,False
2,image_0_thickness_4.png,3,1,"[red, blue]",4,./images/image_0_thickness_4-claude-3-sonnet-20240229-output.md,"In the given image, there is 1 intersection point where the blue line and the red line cross each other.",1,True
3,image_1_thickness_2.png,3,2,"[red, blue]",2,./images/image_1_thickness_2-claude-3-sonnet-20240229-output.md,"In the given image, there are 2 line intersections where the red line intersects with the blue lines, forming two distinct intersection points.",2,True
4,image_1_thickness_3.png,3,2,"[red, blue]",3,./images/image_1_thickness_3-claude-3-sonnet-20240229-output.md,"In the given image, there are 2 line intersections, where the red line intersects the blue line forming a triangle-like shape.",2,True
5,image_1_thickness_4.png,3,2,"[red, blue]",4,./images/image_1_thickness_4-claude-3-sonnet-20240229-output.md,"In the given image, there are 3 line intersections where the red line and the blue lines meet or intersect with each other.",3,False
6,image_2_thickness_2.png,3,1,"[red, blue]",2,./images/image_2_thickness_2-claude-3-sonnet-20240229-output.md,"In the given image, there is 1 intersection point where the blue line crosses the red line.",1,True
7,image_2_thickness_3.png,3,1,"[red, blue]",3,./images/image_2_thickness_3-claude-3-sonnet-20240229-output.md,"In the given image, there is 1 intersection point where the blue line and the red line intersect each other.",1,True
8,image_2_thickness_4.png,3,1,"[red, blue]",4,./images/image_2_thickness_4-claude-3-sonnet-20240229-output.md,"In the image, there is 1 intersection point where the blue line and the red line cross each other.",1,True
9,image_3_thickness_2.png,3,1,"[red, blue]",2,./images/image_3_thickness_2-claude-3-sonnet-20240229-output.md,"There is 1 line intersection in the image, where the red line crosses the blue line, forming an X shape.",1,True
